# Symbolic results 

It is also of interest to work with non-defined variables in order to look for analytical results. This can be done integrating the Fermionic package together with SymPy via PyCall. There are also packages for defining symbolic fermionic operators. In this notebook we will see how exactly does this work.

1. [Symbolic interaction](#symbol)
2. [Symbolic operators](#sops)


<a id="symbol"></a>
## Symbolic Algebra

In this section we will learn to interact with the symbolic tools defined in [SymPy](https://github.com/JuliaPy/SymPy.jl). This library works perfectly within Julia via PyCall. I will show you how:

First of all, if you have not done ir yet, you should install Conda:

`using Pkg
Pkg.add("Conda") 
using Conda
Conda.update()
Pkg.add("PyCall")`

We then call PyCall

In [1]:
using PyCall
sympy = pyimport("sympy");

With this amazing package we can work with symbols as they were variables

In [2]:
x = sympy.Symbol("x")      # PyObject x

PyObject x

In [3]:
sympy.Matrix([x, 2, 3])

PyObject Matrix([
[x],
[2],
[3]])

So, for instance, we can exactly diagonalize the superconducting hamiltonian in a symbolic way:
$H=\sum_k \epsilon_k (a_k^\dagger a_k + a_{\bar{k}}^\dagger a_{\bar{k}})-\sum_{k,k'} G_{k,k'} a_{k'}^\dagger a_{\bar{k'}}^\dagger a_{\bar{k}} a_k$
We will set $\epsilon_k = \epsilon \; \forall k$, and $G_{kk'} = g \; \forall k,k'$.

In [4]:
using Fermionic
using SparseArrays

g = sympy.Symbol("g")
e = sympy.Symbol("e")

d = 4 #this will be a system of 4 levels
nume = Int(d/2)

e0 = 1.0

o = Op_fixed(d, nume)

epsilon = [e0*(i-d/4-1/2) for i in 1:d/2]
epsilon = sort([epsilon; epsilon])
h0 = sum([epsilon[i]*(ada(o,i,i) + ada(o,i+1,i+1)) for i in 1:2:(Int(d)-1)])
hi = sum([sum([if i==j spzeros(binomial(d,nume), binomial(d,nume)) else -(ada(o,j,i+1)*ada(o,j+1,i)) end
                    for i in 1:2:(Int(d)-1)]) for j in 1:2:(Int(d)-1)])

mh = e*sympy.Matrix(h0) - g*sympy.Matrix(hi)
(P, D) = mh.diagonalize();
D #this outputs the diagonal matrix.

PyObject Matrix([
[0, 0, 0, 0,                   0,                  0],
[0, 0, 0, 0,                   0,                  0],
[0, 0, 0, 0,                   0,                  0],
[0, 0, 0, 0,                   0,                  0],
[0, 0, 0, 0, -(e**2 + g**2)**0.5,                  0],
[0, 0, 0, 0,                   0, (e**2 + g**2)**0.5]])

The eigenvalues are the columns of the following matrix:

In [5]:
P

PyObject Matrix([
[  0,   0,   0,   0, g/(e + (e**2 + g**2)**0.5), g/(e - (e**2 + g**2)**0.5)],
[1.0,   0,   0,   0,                          0,                          0],
[  0, 1.0,   0,   0,                          0,                          0],
[  0,   0, 1.0,   0,                          0,                          0],
[  0,   0,   0, 1.0,                          0,                          0],
[  0,   0,   0,   0,                        1.0,                        1.0]])

You can learn more about matrix manipulation in [this tutorial](https://docs.sympy.org/latest/modules/matrices/matrices.html).

<a id="sops"></a>
## Symbolic second quantization

There is a library inside SymPy which allows the definition of fermionic symbolic operators. You can finde the complete list of functions in [this link](https://docs.sympy.org/latest/modules/physics/secondquant.html).

It is good enough for working with commutators, but for some reasons the anticommutator identity is not defined for fermions, making it a little limited for complete usage.

In [6]:
using PyCall
sp = pyimport("sympy");
sq = pyimport("sympy.physics.secondquant");
i,j = sp.symbols("i j", above_fermi=true) #it could also be below_fermi
ai = sq.F(i);
adi = sq.Fd(i);
aj = sq.F(j)
adj = sq.Fd(j)

PyObject CreateFermion(j)

We can evauate contractions:

In [7]:
sq.contraction(adi,adi) #double creation operator

PyObject 0

In [8]:
sq.contraction(aj,aj) #double destruction operator

PyObject 0

In [9]:
a = sq.contraction(adi,aj)

PyObject 0

In [10]:
a = sq.contraction(ai,adj)

PyObject KroneckerDelta(i, j)

In [11]:
sq.evaluate_deltas(a)

PyObject KroneckerDelta(i, j)

Let's check anticommutation relations:

In [12]:
sq.wicks(adi*adj+adj*adi)

PyObject 0

In [13]:
sq.wicks(ai*aj+aj*ai)

PyObject 0

In [14]:
sq.wicks(ai*adj+adj*ai)

PyObject KroneckerDelta(i, j)

We can define commutators

In [15]:
com = sq.Commutator(adi*adj,ai)

PyObject Commutator(CreateFermion(i)*CreateFermion(j),f(i))

and expand them with doit()

In [16]:
com.doit(wicks = true) #wicked defines a normal ordering of terms (see Wick's expantion)

PyObject KroneckerDelta(i, j)*CreateFermion(i) - CreateFermion(j)

Also perform a wick expantion

In [17]:
sq.wicks(adi*(ai+adj))

PyObject NO(CreateFermion(i)*AnnihilateFermion(i)) + NO(CreateFermion(i)*CreateFermion(j))

In [18]:
sq.wicks(ai*(adi+adj), keep_only_fully_contracted = true)

PyObject KroneckerDelta(i, j) + 1

NO() performs a normal order, assuming anticommutation relations

In [19]:
sq.NO(aj*adi)

PyObject -NO(CreateFermion(i)*AnnihilateFermion(j))

In [20]:
adj = sq.Dagger(sq.CreateFermion(sp.Symbol("j")))

PyObject AnnihilateFermion(j)

### Bra and Kets

Each state can only have one particle, so we choose to store a list of occupied orbits rather than a tuple with occupation numbers (zeros and ones).

In [21]:
f1 = sq.FKet([1,2])

PyObject FockStateKet((1, 2))

In [22]:
f2 = sq.FBra([1,3])

PyObject FockStateKet((1, 3))

In [23]:
sq.InnerProduct(f2,f1)

PyObject 0